In [1]:
from thermoclassifier.dataset.dataset_creator import DatasetCreator
from torch.utils.data import DataLoader

In [2]:
dc = DatasetCreator(user='phase', measurement='C', elements=None)
train_dataset, test_dataset, val_dataset = dc.get_datasets()

# Create the DataLoaders
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

Ag 1
Al 2
Am 3
As 4
Au 5
B 6
Ba 7
Be 8
Bi 9
C 10
Ca 11
Cd 12
Ce 13
Co 14
Cr 15
Cs 16
Cu 17
Dy 18
Er 19
Eu 20
Fe 21
Ga 22
Gd 23
Ge 24
Hf 25
Hg 26
Ho 27
In 28
Ir 29
K 30
La 31
Li 32
Lu 33
Mg 34
Mn 35
Mo 36
N 37
Na 38
Nb 39
Nd 40
Ni 41
Np 42
O 43
Os 44
P 45
Pa 46
Pb 47
Pd 48
Pr 49
Pt 50
Pu 51
Rb 52
Re 53
Rh 54
Ru 55
S 56
Sb 57
Sc 58
Se 59
Si 60
Sm 61
Sn 62
Sr 63
Ta 64
Tb 65
Tc 66
Te 67
Th 68
Ti 69
Tl 70
Tm 71
U 72
V 73
W 74
Y 75
Yb 76
Zn 77
Zr 78
Dataset shape:  (20466, 5, 4)


In [3]:
for d in train_loader:
    print(d)
    break

tensor([[[ 451.0000,   28.2385,   21.0000,    3.0000],
         [ 483.0000,   28.9602,   21.0000,    3.0000],
         [ 730.0000,   36.2099,   21.0000,    3.0000],
         [1424.0000,   36.0064,   21.0000,    0.0000],
         [1534.0000,   36.9586,   21.0000,    0.0000]]], dtype=torch.float64)
